In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io as spio
import matplotlib.lines
import scipy.stats as spst
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mlxtend.evaluate import permutation_test

def loadValuesFromFile(inputRawFile):
    imat = spio.loadmat(inputRawFile)
    if 'spaSpikeTrainError' in imat.keys():
        return np.amax(imat.get('spaSpikeTrainError'))
    else:
        return np.amax(imat.get('cueSpikeTrainError'))

def loadNullSpikeTrainValues():
    nullFile = ''
    nmat = spio.loadmat(nullFile)
    return nmat.get('nullSpikeTrainAcc').flatten()

def getAllTrueAccuracyValues(inputPaths):
    outputAcc = []
    for singleRawP in inputPaths:
        for aRoot,aDirs,aFiles in os.walk(singleRawP):
            for aFile in aFiles:
                if aFile.endswith('.mat'):
                    currRawFile = os.path.join(singleRawP,aFile)
                    currAcc = loadValuesFromFile(currRawFile)
                    outputAcc.append(currAcc)
    return outputAcc,loadNullSpikeTrainValues()

def generateFaceColors(allIn,nullArr):
    outC = []
    for val in allIn:
        pVal = permutation_test([val],nullArr,method='approximate',num_rounds=1000,seed=0)
        if pVal < 0.05:
            outC.append('r')
        else:
            outC.append('b')   
    return outC

def generateDataDataframe(inputX):
    labels = np.reshape([1]*len(inputX),(len(inputX),1))
    values = np.reshape(inputX,(len(inputX),1))
    fullValueDF = pd.DataFrame(np.concatenate((labels,values),axis=1),columns=['Loc','Acc'])
    return fullValueDF

def createPlot(inputRawPaths,figurePath):
    trueAccValues,nullAccValues = getAllTrueAccuracyValues(inputRawPaths)
    valDF = generateDataDataframe(trueAccValues)
    #print(np.mean(trueAccValues),np.std(trueAccValues)/np.sqrt(len(trueAccValues)))
    #xScatter = np.random.uniform(-0.2,.2,size=(len(trueAccValues),))
    xScatter = np.asarray([0.08411185,-0.13956483,0.18893066,-0.04586438,-0.00825243,
                           0.11551821,0.11248285,-0.16177019,-0.15676147,0.17467314])
    #faceColors = generateFaceColors(trueAccValues,nullAccValues)
    legElem1 = [Line2D([0],[0],color='white',marker='o',markersize=12,markerfacecolor='b',label='p ≥ 0.05'),
                Line2D([0],[0],color='white',marker='o',markersize=12,markerfacecolor='r',label='p < 0.05'),
               ]
    fig = plt.figure(figsize=(20,20))
    ax1 = fig.add_subplot(2,8,1)
    sns.barplot(x='Loc',y='Acc',data=valDF,palette=['white'],capsize=.1,alpha=0,ax=ax1)
    ax1.scatter(xScatter,trueAccValues,color='k')
    ax1.set_ylim([.8,1])
    ax1.set_xticklabels('')
    ax1.set_xlabel('')
    ax1.set_ylabel('Simulated Spike Train Accuracy',fontsize=16)
    #ax1.legend(handles=legElem1,fontsize=12,loc='lower right')
    #plt.show()
    fig.savefig(figurePath,bbox_inches='tight')
    plt.close()
    return

allRPaths = []
#figSPath = ''
figSPath = ''
figVar = createPlot(allRPaths,figSPath)
print('Done')
